### timer - Hardware timer classes

In real time firmware, hardware timers are necessary and must be as fast as possible.  I've interfaced the ``c`` API for timers in a signal/slot oriented way in ``ah_timer`` as well as simply, in ``simple_timer``.  these use ``sigevents`` and ``itimerspec`` both to generate the timer.

#### Example

Below are two extremely simple examples on how to use my timers.

In [6]:
from cppm import cppmagic

In [10]:
%set_compiler g++ --std=c++11 -Wall -lrt

In [12]:
%%runcpp simple_timer_test
#include <stdio.h>
#include "../common/mach_gettime.h"
#include "../simple_timer.h"
#include "../ah_signal.h"

bool bw_cont = true;
bool ub_cont = true;
simple_timer* ub, *bw;

void usain_bolt(int signum){
    ub->stop();
    printf("Usain Bolt finished 100 meters.\n");
    ub_cont = false;
}

void betty_white(int signum){
    bw->stop();
    printf("Betty White finished 100 meters.\n");
    bw_cont = false;
}

int main(void){
    ub = new simple_timer();
    ah_signal(SIGUSR1, &usain_bolt);
    bw = new simple_timer();
    ah_signal(SIGUSR2, &betty_white);
    ub->start(1.5);
    bw->start(5.0);
    while(bw_cont || ub_cont){
        printf(".");
        sleep(1);
    }
    printf("Time was %f\n",ub->get_t());
    printf("Time was %f\n",bw->get_t());

    return 0;
}

CompilationError: In file included from simple_timer_test.cpp:3:
./../simple_timer.h:27:2: error: unknown type name 'timer_t'; did you mean 'time_t'?
        timer_t timerID;
        ^~~~~~~
        time_t
/usr/include/sys/_types/_time_t.h:30:26: note: 'time_t' declared here
typedef __darwin_time_t         time_t; 
                                ^
In file included from simple_timer_test.cpp:3:
./../simple_timer.h:30:23: error: field has incomplete type 'struct itimerspec'
    struct itimerspec its;
                      ^
./../simple_timer.h:30:12: note: forward declaration of 'itimerspec'
    struct itimerspec its;
           ^
./../simple_timer.h:82:5: error: use of undeclared identifier 'timer_create'; did you mean 'kmod_create'?
    timer_create(CLOCK_REALTIME, &te, &timerID);
    ^~~~~~~~~~~~
    kmod_create
/usr/include/mach/host_priv.h:173:15: note: 'kmod_create' declared here
kern_return_t kmod_create
              ^
In file included from simple_timer_test.cpp:3:
./../simple_t

Now, to connect to the memory.  We have to attach using the same key, which is a little bit of a paint, but if you use macros effectively, it'll just be like pointing at a file.  This client will be way faster than using files though, for time intensive processes, use these ``shm``s.

In [3]:
%%runcpp shm_client
#include "../ah_shm.h"
#include <string>

int main(void){
    shm_client<std::string> *holmes = new shm_client<std::string>(5678);
    for(int i=1; i<5; i++){
        const char* desc = holmes->get().c_str();
        printf("Episode %d in which Sherlock %s\n", i, desc);
        sleep(1);
    }
    delete holmes;
    return 0;
}

Episode 1 in which Sherlock is dead!
Episode 2 in which Sherlock is now alive again!
Episode 3 in which Sherlock is a good guy!
Episode 4 in which Sherlock is a bad guy!



Now, we just have to stop the server to make sure we're not leaving too many things open.

In [4]:
%killall

Just a note:  I'm still working on the cleanup of this.  To check that I've removed all of the attached shared memory, you can use the bash line

In [5]:
!ipcs

IPC status from <running system> as of Wed May 24 10:18:34 EDT 2017
T     ID     KEY        MODE       OWNER    GROUP
Message Queues:

T     ID     KEY        MODE       OWNER    GROUP
Shared Memory:
m  65536 0x52020016 --rw-------     root    wheel
m 196609 0x0000162e --rw-rw-rw-   ahagen    staff

T     ID     KEY        MODE       OWNER    GROUP
Semaphores:



and to remove it, find the ``KEY`` corresponding to it and use ``ipcrm``.  Note that the KEY is usually the hexadecimal representation of the ``key`` you used in ``c++``, so you can just use that:

In [6]:
!ipcrm -M 5678